<a href="https://colab.research.google.com/github/gabrielursulino/ciencia-de-dados-e-analytics/blob/main/MVP/MVP_ANALISE_DE_DADOS_E_BOAS_PRATICAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP Análise de Dados e Boas Práticas
**Nome:** Gabriel Lopes Ursulino

**Matrícula:** 4052025000406

**Dataset:** [Cerebral Stroke Prediction-Imbalanced Dataset](https://www.kaggle.com/datasets/shashwatwork/cerebral-stroke-predictionimbalaced-dataset/data)

# Descrição do Problema


O Acidente Vascular Cerebral (AVC), também conhecido pelo termo em inglês *stroke*, é uma das principais causas de morte e incapacidade no mundo. O AVC acontece quando vasos que levam sangue ao cérebro entopem ou se rompem, provocando a paralisia da área cerebral que ficou sem circulação sanguínea. O AVC pode ser hemorrágico, quando há rompimento de um vaso cerebral, provocando hemorragia, ou isquêmico, quando há obstrução de uma artéria, impedindo a passagem de oxigênio para células cerebrais, que acabam morrendo. Essa obstrução pode acontecer devido a um trombo (trombose) ou a um êmbolo (embolia).

O conjunto de dados *Cerebral Stroke Prediction-Imbalanced Dataset* é um conjunto de dados multivariado que reúne diversas características que podem estar associadas ao risco de AVC, como idade, hábitos, estilo de vida e condições prévias de saúde como hipertensão e doenças cardiovasculares.

O objetivo deste MVP é realizar uma análise exploratória e pré-processamento dos dados para identificar os principais fatores de risco relacionados à ocorrência de AVC.

A partir dessa análise inicial, espera-se fornecer insumos para possíveis modelos preditivos futuros ou estratégias de prevenção.

## Hipóteses do Problema

Tracei as seguintes hipóteses:

*   O nível médio de glicose (avg_glucose_level) influencia no risco de AVC?
*   O IMC (bmi) está relacionado à ocorrência de AVC?
*   O gênero influencia na ocorrência de um AVC?
*   Existe correlação entre hipertensão e a ocorrência de um AVC?
*   O que podemos inferir a respeito do estilo de vida (tabagismo, local de moradia, tipo de emprego)?
*   Qual das características apresenta maior fator de risco com AVC?
*   Como os fatores combinados afetam o risco?


## Tipo de Problema

Este é um problema de classificação supervisionada. Dado um conjunto de características (idade, genero, hipertenso, fumante etc.), o objetivo é avaliar os fatores de risco que contribuem para ocorrência de AVC para prever futuras ocorrências.

## Seleção de Dados

O dataset apresentado é amplamente disponível. Não é necessária uma etapa de seleção de dados externa, pois o dataset já está curado e pronto para uso.

## Atributos do Dataset


O dataset contém 43.400 amostras, com os seguintes atributos:

- ***id*** (Identificador único do paciente)
- ***gender*** (Gênero: "Male", "Female" or "Other"), em português: "masculino", "feminino" ou "outro", respectivamente.
- ***age*** (Idade do paciente em anos)
- ***hypertension*** (hipertensão: 0 se o paciente não possui, 1 se o paciente possui)
- ***heart_disease*** (doença cardiovascular: 0 se o pacitente não possui, 1 se o paciente possui)
- ***ever_married*** (Se o paciente já foi casado: "Yes" or "No") em português : "sim" ou "não", respectivamente.
- ***work_type*** (Tipo de trabalho: se é do setor privado, autônomo, se nunca trabalhou etc.)
- ***Residence_type*** (Local de moradia: "Rural" or "Urban") em português: "rural" ou "urbano", respectivamente.
- ***avg_glucose_level*** (Nível médio de glicose no sangue do paciente em mg/dL)
- ***bmi*** (Body Mass Index, ou Índice de Massa Corporal (IMC))
- ***smoking_status*** (Status de tabagismo, **never smoked** = nunca fumou, **formerly smoked** = ex-fumante, **smokes** = fumante)
- ***stroke*** (AVC: 0 se o paciente não teve AVC, 1 se o paciente teve)

# Importação das Bibliotecas Necessárias e Carga de Dados

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE

In [ ]:
# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/gabrielursulino/ciencia-de-dados-e-analytics/refs/heads/main/datasets/stroke_dataset.csv"

# Lê o arquivo
avc = pd.read_csv(url, delimiter=',')

*Para mais detalhes a respeito deste dataset, acesse:* https://www.kaggle.com/datasets/shashwatwork/cerebral-stroke-predictionimbalaced-dataset/data

#Análise Exploratória de Dados

##Informações Gerais

In [ ]:
# Verificando as dimensões
print(f"Dimensões do Dataset: {avc.shape}")
print("-" * 30)

# Verificando os tipos de dados e valores não-nulos
print("Informações Gerais e Tipos de Dados:")
avc.info()

O dataset que será trabalhado possui 43.400 instâncias e 12 colunas. Temos 3 variáveis do tipo *float*, 4 do tipo *int* e 5 categóricas.

In [ ]:
# Verificando a quantidade de nulos
avc.isnull().sum()

**bmi:** Possui 3,37% de nulos, uma quantidade pequena o suficiente para que possamos preencher os valores sem introduzir um grande viés. Remover as linhas seria uma perda de informação desnecessária.

**smoking_status:** Possui 30.63% de valores ausentes.

In [ ]:
# Verificando a existencia de duplicadas no dataset
duplicates=avc[avc.duplicated()]
print(duplicates)

Além de um resumo da quantidade de instâncias por colunas e os tipos dos dados, podemos notar a presença de valores nulos em *bmi* e *smoking_status*. Não há a presença de duplicadas.

In [ ]:
# Informação em % da distribuição
print("Proporção das classes em 'stroke' em %:")
print(avc['stroke'].value_counts(normalize=True) * 100)

# Gráfico de barras para análise da variável alvo
plt.figure(figsize=(7, 5))
sns.countplot(x='stroke', hue='stroke', data=avc, palette={0: '#ADD8E6', 1: '#FF0000'}, legend=False)
plt.title('Distribuição da ocorrência de AVC')
plt.xlabel('')
plt.ylabel('Contagem')
plt.xticks(ticks=[0, 1], labels=['Não teve AVC (0)', 'Teve AVC (1)'])
plt.show()

Com esse gráfico podemos confirmar que se trata de um dataset extremamente desbalanceado em termos de classes (98,20% contra 1,80%).  Isso exigirá balanceamento para evitar que o modelo seja enviesado para a classe majoritária.

In [ ]:
# Mostra as 10 primeiras linhas do dataset
avc.head(10)

Aqui podemos ter uma visão mais ampla do dataset. É possível notar que também há crianças na base de dados. Na linha de índice 2, podemos notar que existe uma criança de 8 anos com trabalho no setor privado. Embora possa parecer estranho, é possível. Posso citar como exemplo atores mirins de novelas ou filmes. Coluna ID não é relevante e pode ser removida.

In [ ]:
# Mostra as 10 últimas linhas do dataset para atestar que o dataset foi carregado com sucesso
avc.tail(10)

## Estatísticas Descritivas

Resumo estatístico somente das variáveis contínuas.

In [ ]:
# Resumo estatístico do dataset (média, desvio padrão, mínimo, máximo e os quartis)
vcontinuas = ['age', 'avg_glucose_level', 'bmi'] #define quais colunas serão utilizadas
avc[vcontinuas].describe()

Este resumo nos indica que, na média, é uma população com idade de 42 anos e pré-diabética, de acordo com a *American Diabetes Association* e  outras diversas fontes.

***bmi*** (IMC): Trata-se de uma população que, em média, está com sobrepeso. Na mínima vou assumir que é uma pessoa com desnutrição e a máxima que trata-se de uma pessoa com obesidade mórbida. Os quartis indicam que de fato a maioria das pessoas é obesa ou tem sobrepeso.

***age*** (Idade): Temos uma ampla faixa de idade, indo desde bebês até idosos. Isso é identificável pelo desvio padrão, mínima e máxima. Ótimo para analisarmos como AVC se distribui por idade.

No geral, o desvio padrão indica dados bem heterogêneos e os valores máximos indicam a possível presença de outliers em *bmi* e *avg_glucose_level*.

[Clique aqui para conferir o diagnóstico de diabetes.](https://diabetes.org/about-diabetes/diagnosis#:~:text=Understanding%20Diabetes%20Diagnosis%20*%20Diabetes%20is%20diagnosed,greater%20than%20or%20equal%20to%20200%20mg/dl.)

[Clique aqui para conferir a tabela do IMC.](https://www.who.int/europe/news-room/fact-sheets/item/a-healthy-lifestyle---who-recommendations#:~:text=Observa%C3%A7%C3%A3o%20.,24%2C9%20como%20IMC%20normal.)

## Visualizações e Análise de Correlações

###Visualização das demais variáveis categóricas

In [ ]:
# Confirguração e título
fig, axes = plt.subplots(3, 3, figsize=(15, 18))
fig.suptitle('Distribuição das Variáveis Categóricas', fontsize=18, y=1.02)

# Gênero
sns.countplot(ax=axes[0, 0], data=avc, x='gender', hue='gender', palette={'Male': 'skyblue', 'Female': 'lightcoral', 'Other':'grey'}, legend=False)
axes[0, 0].set_title('Distribuição por Gênero', fontsize=12)

# Hipertensão
sns.countplot(ax=axes[0, 1], data=avc, x='hypertension', hue='hypertension', palette={0: 'royalblue', 1: 'crimson'}, legend=False)
axes[0, 1].set_title('Hipertensão', fontsize=12)
axes[0, 1].set_xticks([0, 1])
axes[0, 1].set_xticklabels(['Não Possui (0)', 'Possui (1)'])

# Doença Cardíaca
sns.countplot(ax=axes[0, 2], data=avc, x='heart_disease', hue='heart_disease', palette={0: 'royalblue', 1: 'crimson'}, legend=False)
axes[0, 2].set_title('Doença Cardíaca', fontsize=12)
axes[0, 2].set_xticks([0, 1])
axes[0, 2].set_xticklabels(['Não Possui (0)', 'Possui (1)'])

# Estado Civil
sns.countplot(ax=axes[1, 0], data=avc, x='ever_married', hue='ever_married', palette={'No': 'grey', 'Yes': 'lightgreen'}, legend=False)
axes[1, 0].set_title('Distribuição por "Se já casado"', fontsize=12)

# Tipo de Trabalho
sns.countplot(ax=axes[1, 1], data=avc, x='work_type', hue='work_type', palette='Set2')
axes[1, 1].set_title('Distribuição por Tipo de Trabalho', fontsize=12)
axes[1, 1].tick_params(axis='x', rotation=30) # Rotaciona os rótulos do eixo X

# Status de Tabagismo
sns.countplot(ax=axes[1, 2], data=avc, x='smoking_status', hue='smoking_status', palette={'smokes': 'firebrick', 'never smoked': 'mediumseagreen', 'formerly smoked':'goldenrod'}, legend=False)
axes[1, 2].set_title('Distribuição por Status de Tabagismo', fontsize=12)
axes[1, 2].tick_params(axis='x', rotation=30) # Rotaciona os rótulos do eixo X

# Local de moradia
sns.countplot(ax=axes[2, 0], data=avc, x='Residence_type', hue='Residence_type', palette={'Rural': 'mediumseagreen', 'Urban': 'grey'}, legend=False)
axes[2, 0].set_title('Distribuição por Local de moradia', fontsize=12)
axes[2, 0].tick_params(axis='x', rotation=30) # Rotaciona os rótulos do eixo X

# Formata eixos
for ax in axes.flat:
    ax.set_xlabel('')
    ax.set_ylabel('Contagem')

# Apaga os eixos que não foram usados
for i in range(7, 9):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout(rect=[0, 0, 1, 0.96]) # Ajusta o layout para evitar sobreposição
plt.show()

**Hipertensão e Doença Cardíaca:** Em ambos os gráficos, a quantidade de pessoas que não possuem a condição (0) é esmagadoramente maior do que a quantidade de pessoas que possuem (1). Fica evidente o desbalanceamento das classes.

**Distribuição por Gênero:** Possui um número maior de mulheres do que de homens. A categoria "Other" é praticamente residual.A predominância feminina é um dado demográfico importante. A categoria "Other", por ser tão pequena, pode ser removida na etapa de pré-processamento para não adicionar ruído ao modelo.

**Distribuição por Estado Civil (ever_married):** Há uma proporção significativamente maior de pessoas que já foram casadas ('Yes') em comparação com as que nunca se casaram ('No').

**Distribuição por Tipo de Trabalho (work_type):** A categoria "Private" (trabalho no setor privado) domina a amostra. "Self-employed" (autônomo) é a segunda mais comum, mas com uma contagem muito menor. A grande concentração em "Private" mostra que a amostra não é uniformemente distribuída entre os setores de trabalho. A categoria "children" confirma o que há uma quantidade relevante de crianças no dataset. Já a categoria "Never_worked" é rara.

**Distribuição por Status de Tabagismo (smoking_status):** A maioria dos indivíduos na amostra nunca fumou. As categorias "formerly smoked" (ex-fumante) e "smokes" (fumante) representam grupos menores, mas ainda muito relevantes. Diferente de hipertensão e doença cardíaca, aqui temos uma boa representatividade dos três grupos principais.

**Distribuição por Local de moradia:** Esta é a variável melhor distribuída entre as categorias.

###Categóricas x Ocorrência de AVC

In [ ]:
# Confirguração e título
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
plt.suptitle('Variáveis Categóricas vs. Ocorrência de AVC', fontsize=18)

# Gênero
sns.countplot(ax=axes[0, 0], x='gender', hue='stroke', data=avc, palette={0: 'lightblue', 1: 'red'})
axes[0, 0].set_title('Distribuição por Gênero', fontsize=12)

# Hipertensão
sns.countplot(ax=axes[0, 1], x='hypertension', hue='stroke', data=avc, palette={0: 'lightblue', 1: 'red'})
axes[0, 1].set_title('Hipertensão', fontsize=12)
axes[0, 1].set_xticks([0, 1])
axes[0, 1].set_xticklabels(['Não Possui (0)', 'Possui (1)'])

# Doença Cardíaca
sns.countplot(ax=axes[0, 2], x='heart_disease', hue='stroke', data=avc, palette={0: 'lightblue', 1: 'red'})
axes[0, 2].set_title('Doença Cardíaca', fontsize=12)
axes[0, 2].set_xticks([0, 1])
axes[0, 2].set_xticklabels(['Não Possui (0)', 'Possui (1)'])

# Estado Civil
sns.countplot(ax=axes[1, 0], x='ever_married', hue='stroke', data=avc, palette={0: 'lightblue', 1: 'red'})
axes[1, 0].set_title('Distribuição por "Se já casado"', fontsize=12)

# Tipo de Trabalho
sns.countplot(ax=axes[1, 1], x='work_type', hue='stroke', data=avc, palette={0: 'lightblue', 1: 'red'})
axes[1, 1].set_title('Distribuição por Tipo de Trabalho', fontsize=12)
axes[1, 1].tick_params(axis='x', rotation=30) # Rotaciona os rótulos do eixo X

# Status de Tabagismo
sns.countplot(ax=axes[1, 2], x='smoking_status', hue='stroke', data=avc, palette={0: 'lightblue', 1: 'red'})
axes[1, 2].set_title('Distribuição por Status de Tabagismo', fontsize=12)
axes[1, 2].tick_params(axis='x', rotation=30) # Rotaciona os rótulos do eixo X

# Local de moradia
sns.countplot(ax=axes[2, 0], x='Residence_type', hue='stroke', data=avc, palette={0: 'lightblue', 1: 'red'})
axes[2, 0].set_title('Distribuição por Local de moradia', fontsize=12)
axes[2, 0].tick_params(axis='x', rotation=30) # Rotaciona os rótulos do eixo X

# Formata legenda e eixos
for ax in axes.flat:
    ax.set_xlabel('')
    ax.set_ylabel('Contagem')

    # Substitui a legenda de cada subplot, se ela existir
    legenda = ax.get_legend()
    if legenda is not None:
        legenda.set_title('AVC')
        legenda.get_texts()[0].set_text('Não')
        legenda.get_texts()[1].set_text('Sim')

# Apaga os eixos que não foram usados
for i in range(7, 9):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout(rect=[0, 0, 1, 0.96]) # Ajusta o layout para evitar sobreposição
plt.show()

Para interpretar esses gráficos é importante não olhar para a altura absoluta das barras vermelhas (que sempre serão pequenas devido ao desbalanceamento), mas sim para a proporção da barra vermelha em relação à sua correspondente barra azul.

**Gênero (gender):** O AVC parece ocorrer mais em homens que em mulheres. Poderemos confirmar isso mais adiante olhando para os números.

**Hipertensão (hypertension) e Doença Cardíaca (heart_disease):** A proporção de pessoas com AVC é visivelmente maior entre aqueles que têm hipertensão ou doença cardíaca. São fortes indicadores.

**Casamento (ever_married):** A proporção de AVC é maior entre as pessoas que já foram casadas. No entanto, é intuitivo pensar que se a pessoa já foi casada ou não pode ter correlação com a idade.

**Tabagismo (smoking_status):** Fumantes (smokes) e ex-fumantes (formerly smoked) apresentam uma taxa de AVC ligeiramente maior do que os não-fumantes.  O fato de "formerly smoked" também apresentar um risco elevado é muito relevante. Isso sugere que os efeitos danosos do tabagismo podem ser duradouros e que parar de fumar, embora benéfico, não elimina imediatamente todo o risco acumulado. Aparentemente, tanto o histórico de tabagismo quanto o hábito atual são fatores de risco importantes.

**Tipo de trabalho (work_type):** O gráfico indica uma ocorrência maior em "self-employed" (autônomo). Analisaremos os números adiante

**Local de moradia (Residence_type):** O gráficos indica não ser um  indicador forte para ocorrência de AVC.

In [ ]:
# Agrupando por tipo de trabalho e calculando a ocorrência de AVC
taxa_avc_por_trabalho = avc.groupby('work_type')['stroke'].mean() * 100
print(taxa_avc_por_trabalho.round(2))

Através dessa análise, podemos concluir que a ocorrência de AVC é duas vezes mais frequente em trabalhadores autônomos (Self-employed), quando comparamos com trabalhadores do setores privado (Private) e público (Govt_job).

In [ ]:
# Agrupando por gênero e calculando a ocorrência de AVC
taxa_avc_por_genero = avc.groupby('gender')['stroke'].mean() * 100
print(taxa_avc_por_genero.round(2))

**Nesse dataset**, analisando diretamente os números, mesmo que a ocorrência em homens seja um pouco mais frequente, a diferença é tão pequena que não considero existir uma associação significativa, portanto, não é possível concluir que o gênero é um fator de risco.

In [ ]:
# Boxplot "já casado" por idade
plt.figure(figsize=(10, 6))
sns.boxplot(x='ever_married', y='age', data=avc, hue="ever_married", palette={'No': '#ADD8E6', 'Yes': '#FF0000'}, legend=False)
plt.title('Se "Já casado" por idade')
plt.xlabel('')
plt.ylabel('Idade (anos)')
plt.xticks(ticks=['No', 'Yes'], labels=['Não', 'Sim'])
plt.show()

Com a visualização acima podemos confirmar que se o paciente é ou já foi casado realmente está associado à idade. Dessa forma, a variável "ever_married", funciona como uma representante indireta de idade.

### Histogramas

In [ ]:
# Confirguração e título
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plt.suptitle('Histograma das Variáveis Contínuas', fontsize=16)

# Histograma 1: age (idade)
sns.histplot(ax=axes[0], data=avc, x='age', kde=True, color='lightgreen')
axes[0].set_ylabel('Frequência')
axes[0].set_xlabel('Distribuição de Idade')

# Histograma 2: avg_glucose_level (nível de glicose)
sns.histplot(ax=axes[1], data=avc, x='avg_glucose_level', kde=True, color='lightblue')
axes[1].set_ylabel('Frequência')
axes[1].set_xlabel('Nível Médio de Glicose')

# Histograma 3: bmi (IMC)
sns.histplot(ax=axes[2], data=avc, x='bmi', kde=True, color='red')
axes[2].set_ylabel('Frequência')
axes[2].set_xlabel('IMC')

plt.show()

**Idade (esquerda):** Podemos notar um gráfico multimodal, com picos em idades menores, depois um outro pico entre os 40 e 60 anos e um novo pico próximo aos 80 anos. Isso revela dados não homogêneos. Esse conjunto de dados é composto por grupos distintos de idade, não seguindo uma distribuição normal.

**Nível Médio de Glicose (centro):** Podemos notar um gráfico bimodal, com um pico próximo a 100 e outro próximo de 210. Existe também uma forte assimetria à direita, onde podemos também identificar a presença de outliers no conjunto de barras mais distante do corpo principal.

**IMC (direita):** Um gráfico unimodal, com assimetria à direita e presença de outliers nos valores mais extremos.

Em resumo, os gráficos nos mostram o que foi constatado ainda na etapa das estatísticas descritivas: dados heterogêneos, com vasta distribuição de idade, individos com nível de glicose mais altos e IMC que tende para o sobrepeso e obesidade.

A heterogeneidade observada, especialmente as distribuições não normais e a presença de outliers, são fatores críticos que serão considerados na etapa de pré-processamento e preparação dos dados para algoritmos de machine learning, que podem exigir a transformação dessas variáveis.

###Boxplot

Nas visualizações abaixo, tentaremos encontrar correlações entre as variáveis contínuas e a ocorrência de AVC.

In [ ]:
# Confirguração e título
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plt.suptitle('Variáveis Contínuas vs. Ocorrência de AVC', fontsize=16)

# Boxplot 1: age (idade)
sns.boxplot(ax=axes[0], hue='stroke', x='stroke', y='age', data=avc, palette={0: 'lightblue', 1: 'red'}, legend=False)
axes[0].set_xlabel('')
axes[0].set_ylabel('Idade')
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['Não teve AVC', 'Teve AVC'])

# Boxplot 2: avg_glucose_level (nível de glicose)
sns.boxplot(ax=axes[1], hue='stroke', x='stroke', y='avg_glucose_level', data=avc, palette={0: 'lightblue', 1: 'red'}, legend=False)
axes[1].set_xlabel('')
axes[1].set_ylabel('Nível de Glicose')
axes[1].set_xticks([0, 1])
axes[1].set_xticklabels(['Não teve AVC', 'Teve AVC'])

# Boxplot 3: bmi (IMC)
sns.boxplot(ax=axes[2], hue='stroke', x='stroke', y='bmi', data=avc, palette={0: 'lightblue', 1: 'red'}, legend=False)
axes[2].set_xlabel('')
axes[2].set_ylabel('IMC')
axes[2].set_xticks([0, 1])
axes[2].set_xticklabels(['Não teve AVC', 'Teve AVC'])

plt.show()


Os gráficos revelam uma hierarquia muito clara na força de associação de cada variável com a ocorrência de um AVC. A diferença entre os grupos "Teve AVC" e "Não teve AVC" é drástica para Idade e Glicose, e bem mais sutil para o IMC.

**Idade:** A diferença entre os dois grupos é muito alta. A caixa (representando o intervalo interquartil, ou seja, os 50% centrais dos dados) do grupo "Teve AVC" está quase inteiramente acima da caixa do grupo "Não teve AVC". A mediana da idade (a linha no meio da caixa) de quem teve um AVC está por volta dos 70 anos. A mediana da idade de quem não teve é muito menor, por volta dos 40-45 anos. A idade não é apenas um fator de risco, ela parece ser o fator preditivo mais forte entre os três. Uma pessoa que sofreu um AVC neste conjunto de dados é, tipicamente, mais velha. A separação entre as distribuições é tão clara que a idade, por si só, já parece ser uma variável poderosa para qualquer modelo de previsão.

**Nível de Glicose:** Assim como na idade, há uma separação muito significativa entre os dois grupos. A mediana do nível de glicose no grupo "Teve AVC" é um pouco mais alta (talvez em torno de 110), enquanto no grupo "Não teve AVC" a mediana está na faixa normal (em torno de 95-100). A caixa do grupo "Teve AVC" é também muito mais "alta" (maior variabilidade), indicando que os níveis de glicose entre os pacientes que sofreram AVC variam mais. Níveis elevados de glicose estão fortemente associados à ocorrência de AVC. A grande quantidade de outliers no grupo "Não teve AVC" mostra que existem pessoas com glicose alta que não tiveram AVC, mas a tendência central do grupo que sofreu um AVC é inegavelmente de hiperglicemia.

**IMC:** Aqui, a diferença é bem mais sutil. A mediana do IMC no grupo "Teve AVC" é ligeiramente maior do que no grupo "Não teve AVC" (algo como 30 vs. 28).
Há uma enorme sobreposição entre as duas caixas. Isso significa que muitas pessoas que não tiveram AVC possuem um IMC similar ou até maior do que muitas pessoas que tiveram. Sozinho, o IMC não parece ser capaz de diferenciar bem os dois grupos. Isso sugere que, embora a obesidade seja um fator de risco conhecido, seu impacto pode ser menos direto ou pode ser mais relevante quando combinado com outros fatores (como hipertensão ou idade avançada).

As visualizações confirmam a análise anterior. Os outliers de "nível de glicose" e "IMC" no grupo "Não teve AVC" confirmam perfeitamente as distribuições com cauda longa à direita que vimos nos histogramas. O boxplot agora adiciona o contexto de que esses valores extremos não são tão comuns no grupo que efetivamente sofeu o AVC.

### Facet Grids

O objetivo das visualizações abaixo é entender como os fatores combinados afetam o risco.

In [ ]:
# Idade vs Nível de Glicose
sns.relplot(data=avc, x='age', y='avg_glucose_level', hue='stroke', col='heart_disease', row='hypertension', palette={0: 'royalblue', 1: 'crimson'}, height=5)
plt.suptitle('Idade vs. Glicose, Segmentado por Doença Cardíaca e Hipertensão', y=1.03)
plt.show()

Acima podemos observar a demonstração visual do efeito multiplicador do risco. Ter múltiplos fatores de risco não apenas soma, mas multiplica o perigo. A densidade de pontos vermelhos (AVCs) aumenta dramaticamente à medida que descemos e vamos para a direita na grade.

**Canto Superior Esquerdo:** É a população de menor risco, sem as duas principais comorbidades cardíacas. É uma grande nuvem de pontos, majoritariamente azuis. Os AVCs (pontos vermelhos) ocorrem, mas de forma relativamente esparsa. Mesmo neste grupo sem as comorbidades, os pontos vermelhos não estão distribuídos aleatoriamente. Eles se concentram claramente na parte superior e direita do gráfico. Isso nos mostra que, mesmo sem hipertensão ou doença cardíaca, a idade avançada e um nível de glicose elevado são, por si só, fatores de risco primários e independentes para um AVC.

**Superior Direito e Inferior Esquerdo:** Pessoas com apenas uma das condições: Doença Cardíaca (superior direito) ou Hipertensão (inferior esquerdo).A densidade de pontos vermelhos em ambos os quadrantes é visivelmente maior do que no cenário base. O "vermelho" começa a se destacar mais em meio ao "azul". Adicionar apenas um desses fatores de risco já é suficiente para aumentar significativamente a probabilidade de um AVC em todos os níveis de glicose e todas as faixas de idade onde uma das condições é mais presente.

**Canto Inferior Direito:** O grupo de maior risco acumulado (com hipertensão e doença cardíaca). Este é o quadrante com a menor quantidade de pessoas, mas com a maior proporção de pontos vermelhos para azuis. A densidade de AVCs aqui é altíssima. Neste grupo, os AVCs parecem acontecer de forma mais generalizada. Isso sugere que a combinação de hipertensão e doença cardíaca aumenta o risco, fazendo com que um AVC possa ocorrer mesmo em condições que não seriam suficientes para causá-lo nos outros grupos.

**Risco Independente:** Idade e Glicose são fatores de risco fundamentais, mesmo na ausência de outras comorbidades.

**Risco Aditivo:** Adicionar Hipertensão ou Doença Cardíaca individualmente aumenta notavelmente a densidade de ocorrência de AVC.

**Risco Multiplicador:** A combinação de hipertensão e doença cardíaca cria um cenário de risco extremo, onde a probabilidade de um AVC é drasticamente maior e pode ocorrer em condições de idade e nível de glicose menos severas.

In [ ]:
# age (Idade) vs bmi (IMC)
sns.relplot(data=avc, x='age', y='bmi', hue='stroke', col='heart_disease', row='hypertension', palette={0: 'royalblue', 1: 'crimson'}, height=5)
plt.suptitle(' age (Idade) vs. bmi (IMC), Segmentado por Doença Cardíaca e Hipertensão', y=1.03)
plt.show()

Visualmente, o bmi (IMC) não mostra uma relação tão clara e direta com a ocorrência de AVC quanto o nível de glicose e a idade. Enquanto na análise anterior os pontos vermelhos (AVCs) se concentravam em idades e níveis de glicose elevados, aqui eles parecem ocorrer em quase toda a faixa de IMC, do mais baixo ao mais alto.

**Canto Superior Esquerdo:** Novamente, este é o grupo mais populoso e de menor risco. Os pontos vermelhos são esparsos e aparecem em pessoas com IMC 20, 30, 40 e até acima de 50. Não há uma "zona de perigo" clara para o IMC. O único padrão nítido é que os pontos vermelhos estão quase todos à direita, ou seja, em idades avançadas. Para este grupo, o risco de AVC parece ser uma função quase exclusiva da idade, independentemente do peso.

**Superior Direito e Inferior Esquerdo:** Como antes, a densidade de AVCs aumenta quando adicionamos hipertensão ou doença cardíaca. Mesmo com o risco geral aumentando, o padrão do IMC permanece o mesmo. Os AVCs continuam a ocorrer em toda a gama de IMCs. Uma pessoa com 70 anos e hipertensão parece ter um risco elevado de AVC, seja seu IMC 25 (peso normal/sobrepeso) ou 40 (obesidade severa). Isso sugere que, na presença de uma condição cardíaca grave, o IMC pode ter um papel menos decisivo.

**Canto Inferior Esquerdo:** No grupo de risco máximo, a proporção de pontos vermelhos é a mais alta, e eles aparecem em idades a partir dos 50-60 anos. Mesmo neste cenário extremo, não vemos os AVCs se concentrarem apenas em pessoas com obesidade. Há uma quantidade significativa de pontos vermelhos em IMCs abaixo de 30. A conclusão é que quando um indivíduo acumula os fatores de risco de idade avançada, hipertensão e doença cardíaca, o risco de AVC se torna tão elevado que parece independer do seu peso corporal. Os outros fatores já são potentes o suficiente para causar o evento.

**Comparação com a Análise Anterior:** Idade continua sendo o fator dominante. Em todos os quatro cenários, o fator visual mais forte para prever um AVC é o avanço da idade.

**IMC tem um efeito visualmente disperso:** Diferente do nível de glicose, onde os AVCs se concentravam claramente em valores altos, os AVCs aqui ocorrem em pessoas com baixo peso, sobrepeso e todos os graus de obesidade.

Esta análise, combinada com a anterior, sugere uma hierarquia de importância dos fatores de risco:

**Risco Máximo:** Idade, Hipertensão, Doença Cardíaca.

**Risco Forte:** Nível de Glicose Elevado.

**Risco Menos Claro/Disperso:** IMC.

###Matriz de correlação

In [ ]:
# mapa de calor
categoricas2 = ['age', 'avg_glucose_level', 'bmi', 'heart_disease', 'hypertension', 'stroke'] #define quais colunas serão utilizadas
plt.figure(figsize=(8, 6))
sns.heatmap(avc[categoricas2].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação das Características do dataset AVC')
plt.show()

Embora possa parecer estranho analisar as correlações entre variáveis contínuas e categóricas nessa matriz, não é incorreto e o resultado é uma métrica válida chamada Correlação Ponto-Bisserial. Na matriz acima, as correlações parecem fracas, mas isso deve à natureza dos dados.

O desbalanceamento é o fator principal. Com aproximadamente 98% dos dados sendo "stroke = 0", a tendência linear acaba por ser "diluída". A grande quantidade de pontos na linha Y=0 impede a formação de uma linha de tendência forte, resultando em um coeficiente de correlação mais baixo.

Outro ponto importante é que, como variável stroke só pode ser 0 ou 1 (binária), é matematicamente impossível que a correlação atinja valores muito altos (próximos de 1).

Interessante notar como a idade se relaciona com as demais comorbidades, como hipertensão (hypertension) e doenças cardíacas (heart_disease).

Mesmo sendo um valor baixo, a matriz está alinhada com o que vimos nas análises anteriores. Vemos que idade é o principal fator de risco com correlação 0.16, seguida por doença cardíaca (0.11), logo após temos o nível de glicose e hipertensão (0.08). No entanto, concluo que a correlação linear é uma métrica limitada para este problema.

#Pré-processamento


In [ ]:
# Criando uma cópia para o pré-processamento
avcproc = avc.copy()

In [ ]:
# Verificando cópia do dataset
avcproc.head()

In [ ]:
# Verificando dimensões do dataset cópia
avcproc.shape

Acima foi criada uma cópia do dataframe original para evitar mudanças, facilitando a comparação, experimentação e mantendo a segurança e reversibilidade. Em seguida, pode-se verificar aque a cópia foi realizada corretamente.

## Limpeza

###Tratamento de valores ausentes (Missing values)

Na etapa de Análise Exploratória foi identificada a necessidade de tratamento dos dados ausentes nas colunas ***bmi*** e ***smoking_status***.

A coluna ***bmi*** apresentou valores nulos em aproximadamente 3.37% dos registros e a análise dos boxplots revelou a presença de outliers. A estratégia de imputação escolhida foi preencher estes valores ausentes com a mediana, por ser uma medida mais robusta a valores extremos e, portanto, a mais indicada para não distorcer a distribuição original dos dados.

A coluna ***smoking_status*** apresentou uma quantidade massiva de linhas vazias, correspondendo a mais de 30% do dataset. Diante de um volume tão grande, a imputação com a moda foi descartada por introduzir um viés significativo nos dados.  A exclusão das linhas acarretaria em uma grande perda de informação e a exclusão da coluna também não foi considerada, visto que identificamos o status de tabagismo como um fator de risco para ocorrência de AVC.

Logo, a decisão estratégica foi imputar os valores vazios desta coluna com uma nova categoria, **'unknown'**. Essa abordagem tem a dupla vantagem de preservar todas as instâncias do dataset e, ao mesmo tempo, transformar a falta de informação em uma feature que o próprio modelo de Machine Learning poderá usar para determinar se a ausência de dados sobre tabagismo é, por si só, um fator preditivo.

In [ ]:
# Relembrando a quantidade de valores ausentes
avc.isnull().sum()

In [ ]:
# Substitui valores ausentes por 'unknown' em smoking_status na copia do dataset
avcproc['smoking_status'] = avcproc['smoking_status'].fillna('unknown')

In [ ]:
# Substitui valores ausentes pela mediana em bmi na copia do dataset
median_value = avcproc['bmi'].median()
avcproc['bmi'] = avcproc['bmi'].fillna(median_value)

In [ ]:
# Verificando se alterações foram aplicadas ao somar a quantidade de nulos por colunas
avcproc.isnull().sum()

Alterações aplicadas com sucesso.

###Tratamento de Outliers

Embora tenha sido localizada a presença de diversos outliers, principalmente em **avg_glucose_level** (Nível de Glicose) e **bmi** (IMC), não se tratam de erros de digitação ou medição. São medidas válidas e extremamente valiosas para o modelo de predição. Por essa razão, a decisão estratégica é não fazer nenhum tipo de tratamento e manter os dados originais.

## Feature Engineering e Feature Selection

Durante a etapa de Análise Exploratória, foi identificada que a coluna ID não possui qualquer relevância e que o atributo **"*Other*"** na coluna ***gender*** (gênero) ocorre em pouquíssimas instâncias (0,025%) e não possui qualquer associação com ocorrência de AVC. Por essas razões, os itens mencionados serão descartados.


In [ ]:
# Remove a coluna 'id'
avcproc.drop('id', axis=1, inplace=True)

# Remove a linha com gênero "Other"
avcproc = avcproc[avcproc['gender'] != 'Other']

Como os modelos de Machine Learning são funções matemáticas que operam com número, é fundamental converter todas as variáveis categóricas para um formato numérico. Abaixo utilizaremos a técnica de one-hot encoding para transformar a variáveis categóricas ***work_type*** e ***smoking_status*** em valores binários. Essa abordagem cria novas colunas binárias para cada categoria, evitando a criação de uma falsa relação de ordem que poderia confundir o modelo. Para as demais variáveis categóricas (***gender***, ***ever_married*** e ***Residence_type***) um mapeamento direto para 0 e 1 é a abordagem mais eficiente e direta para realizar a transformação.

In [ ]:
# Transformação de gender, ever married e Residence_type em variáveis binárias
avcproc['gender'] = avcproc['gender'].map({'Male': 0, 'Female': 1})
avcproc['ever_married'] = avcproc['ever_married'].map({'No': 0, 'Yes': 1})
avcproc['Residence_type'] = avcproc['Residence_type'].map({'Rural': 0, 'Urban': 1})

# Aplicando One-Hot Encoding para work_type e smoking_status
avcproc = pd.get_dummies(avcproc, columns=['work_type', 'smoking_status'], drop_first=True, dtype=int)

avcproc.head()

In [ ]:
# Verificando dimensões do dataset
avcproc.shape

Acima podemos verificar a dimensão do dataset após as transformações. A quantidade de colunas aumentou devido a utilização da técnica de *one-hot encoding* e a quantidade de linhas diminuiu em 11 por conta da exclusão das instâncias contendo o atributo **"Other"**.

##Separação entre Treino e Teste e Balanceamento

Etapa fundamental para treinamento do modelo de Machine Learning, no código abaixo optei em dividir o conjunto entre 80/20, garantindo ao modelo um volume de dados robusto e suficiente para aprender os complexos padrões subjacentes aos fatores de risco de um AVC e fornecendo uma avaliação estatisticamente confiável e estável da performance do modelo em dados futuros.

In [ ]:
# Separação dos Dados (Features e Target)
X = avcproc.drop('stroke', axis=1)
y = avcproc['stroke']

# Divisão em Conjuntos de Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

print("Shape de X_train:", X_train.shape)
print("Shape de X_test:", X_test.shape)

A análise exploratória revelou um severo desbalanceamento de classes no dataset, onde a classe minoritária (stroke=1) representa apenas 1,8% do total de registros. Diante deste cenário, é necessário utilizar alguma técnica de balanceamento para evitar um modelo com acurácia falsamente alta prevendo somente a não ocorrência de AVC para todos os pacientes. Optei pelo SMOTE porque ele cria novos dados sintéticos, enriquecendo o dataset e ajudando o modelo a criar uma região de decisão mais ampla e generalista para a classe minoritária, reduzindo o risco de overfitting.

In [ ]:
# Balanceamento de Classes usando SMOTE no conjunto de treino somente
print("Distribuição de classes antes do SMOTE:", y_train.value_counts())

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("\nDistribuição de classes depois do SMOTE:", y_train_resampled.value_counts())

Acima podemos ver que agora as classes estão balanceadas no conjunto de treino.

##Padronização

Devido à existência de outliers e às distribuições assimétricas, optei pela padronização. A normalização comprime as distribuições e um único outlier poderia achatar a escala dos dados restantes. Com a padronização eu evito esse problema, pois ela é mais robusta aos outliers ao utilizar média e desvio padrão para cálculo, ao invés de mínimo e máximo.

In [ ]:
# Instancia o scaler
scaler = StandardScaler()

In [ ]:
# Padronizando as variáveis contínuas age, avg_glucose_level e bmi
# Treina e transforma o conjunto de treino
X_train_resampled[vcontinuas] = scaler.fit_transform(X_train_resampled[vcontinuas])

# Apenas transforma o conjunto de teste com o scaler já treinado
X_test[vcontinuas] = scaler.transform(X_test[vcontinuas])

In [ ]:
#Exibe as primeiras linhas dos dados padronizados
X_train_resampled.head()

In [ ]:
# Confirguração e título dos histogramas padronizados
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plt.suptitle('Histograma das Variáveis Contínuas Padronizadas', fontsize=16)

# Histograma padronizado 1: age (idade)
sns.histplot(ax=axes[0], data=X_train_resampled, x='age', kde=True, color='lightgreen')
axes[0].set_ylabel('Frequência')
axes[0].set_xlabel('Distribuição de Idade')

# Histograma padronizado 2: avg_glucose_level (nível de glicose)
sns.histplot(ax=axes[1], data=X_train_resampled, x='avg_glucose_level', kde=True, color='lightblue')
axes[1].set_ylabel('Frequência')
axes[1].set_xlabel('Nível Médio de Glicose')

# Histograma padronizado 3: bmi (IMC)
sns.histplot(ax=axes[2], data=X_train_resampled, x='bmi', kde=True, color='red')
axes[2].set_ylabel('Frequência')
axes[2].set_xlabel('IMC')

plt.show()

Os histogramas mostram que a forma geral da distribuição foi preservada e os valores foram transformados para ter uma média próxima de zero e um desvio padrão de um, o que era esperado após a padronização.

#Conclusão

A análise exploratória e o pré-processamento do dataset trabalhado forneceram muitas oportunidades e insights interessantes. As diversas visualizações e conclusões estatísticas revelaram correlações importantes e se demonstraram extremamente eficazes para validar as hipóteses levantadas. Conclui-se que o perfil de maior risco que emerge dos dados é o de um indivíduo mais velho, com histórico de hipertensão e/ou doença cardíaca, possui um nível de glicose no sangue mais elevado e que fuma ou já fumou. Devido ao estado e desbalanceamento severo do dataset, foi necessário aplicar diversas técnicas de pré-processamento de dados, desde a limpeza e transformação até o balanceamento e padronização. Todas essas etapas asseguram que o modelo aprenderá com eficiência com as relações presentes nos dados.

As hipóteses levantadas foram respondidas com sucesso.

- *O nível médio de glicose (avg_glucose_level) influencia no risco de AVC?*    Sim, através da análise no bloxplot e do facet grid, podemos ver clara associação entre os níveis de glicose mais altos e a ocorrência de AVC.
- *O IMC (bmi) elevado está relacionado à ocorrência de AVC?* Sozinho, o IMC não demonstrou ser um fator de risco, seu impacto é menos direto e pode ser mais relevante quando combinado com outros fatores.
- *O gênero influencia na ocorrência de um AVC?* Através de uma análise detalhada, com base nos dados aqui estudados, a diferença entre a ocorrência de AVC por gênero se apresentou estatisticamente irrelevante, sendo possível afirmar que um gênero não corre mais risco que outro.
- *Existe correlação entre hipertensão e a ocorrência de um AVC?* Certamente. A ocorrência de AVC em indivíduos hipertensos é consideravemente maior que em indivíduos não hipertensos. A análise comparativa e o facet grid são a confirmação visual dessa conclusão.
- *O que podemos inferir a respeito do estilo de vida (tabagismo, local de moradia, tipo de emprego)?* O indivíduo fumante ou que já fumou apresenta risco consideravelmente mais alto que o indivíduo que nunca fumou. O local de moradia não apresentou qualquer associação com a ocorrência de AVC e o trabalhador autônomo mostrou um risco preocupantemente mais alto que os demais, sugerindo que fatores associados a essa modalidade como estresse ou falta de uma jornada de trabalho bem estabelecidas podem ser um campo para investigações futuras.
- *Qual das características apresenta maior fator de risco com AVC?* Sem dúvida alguma, a idade. Logo nas primeiras visualizações ficou evidente que idade era o principal fator de risco.
- *Como os fatores combinados afetam o risco?* A idade por si só já é um fator de risco importante, mas quando combinada com hipertensão, doença cardíaca e IMC, é criado um cenário de risco extremo.